In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import DecimalType
import pyspark
import pyspark.sql.functions as f
import datetime

packages = ','.join([
    'com.redislabs:spark-redis_2.12:3.1.0'
])

spark = SparkSession \
        .builder \
        .appName("projeto-final-pmd-pedro-jean") \
        .config("spark.redis.host", "redis") \
        .config("spark.redis.port", "6379") \
        .config("spark.redis.auth", "123") \
        .config('spark.jars.packages', packages) \
        .getOrCreate()

In [ ]:
initDate = datetime.datetime(2022, 4, 12).strftime("%Y-%m-%d")

df = spark.read.options(header='True', inferSchema='True', quote="\"", escape="\"").csv('./work/Redis/day24.csv')

treated = df.withColumn('currency', f.when((df.money.isNotNull()) & (df.money != '0'), f.regexp_replace('money', r'(\D*)(\d+(\.|,)?\d*)', '$1')).otherwise(None)) \
    .withColumn('money',  f.regexp_replace('money', r'(\D*)(\d+(\.|,)?\d*)', '$2')) \
    .withColumn('donated', (f.col('money') != '0')) \
    .withColumn('date', f.to_date(f.lit(initDate))) \
    .withColumn('id', f.expr("uuid()"))

treated.show()

In [ ]:
treated.write.format("org.apache.spark.sql.redis") \
    .option("table", "mensagens") \
    .option("key.column", "id") \
    .mode("overwrite") \
    .save()